In [1]:
# Load SQL Magic extension
%load_ext sql

# Import Required Python libraries
import csv, sqlite3
import prettytable
prettytable.DEFAULT = 'DEFAULT'


In [2]:
# Connect to SQLite Database
con = sqlite3.connect("my_data1.db")
cur = con.cursor()

# connects the SQL magic commands to this database
%sql sqlite:///my_data1.db


In [3]:
import pandas as pd

# Load the Dataset into Pandas
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")

# Write DataFrame to SQL Table
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False, method="multi")


101

In [4]:
# Drop the Old Filtered Table (if exists) and Create a New Filtered Table

In [5]:
%sql DROP TABLE IF EXISTS SPACEXTABLE;

%sql CREATE TABLE SPACEXTABLE AS SELECT * FROM SPACEXTBL WHERE Date IS NOT NULL;

 * sqlite:///my_data1.db
Done.
 * sqlite:///my_data1.db
Done.


[]

In [6]:
# Get Unique Launch Sites

In [7]:
%sql SELECT DISTINCT "Launch_Site" FROM SPACEXTABLE;

 * sqlite:///my_data1.db
Done.


Launch_Site
CCAFS LC-40
VAFB SLC-4E
KSC LC-39A
CCAFS SLC-40


In [8]:
# Select 5 records where Launch Site starts with 'CCA'

In [9]:
%sql SELECT * FROM SPACEXTABLE WHERE "Launch_Site" LIKE 'CCA%' LIMIT 5;

 * sqlite:///my_data1.db
Done.


Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of Brouere cheese",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


In [10]:
# Total Payload Mass for NASA (CRS)

In [11]:
%sql SELECT SUM("Payload_Mass__kg_") FROM SPACEXTABLE WHERE "Customer" = 'NASA (CRS)';

 * sqlite:///my_data1.db
Done.


SUM(Payload_Mass__kg_)
45596


In [12]:
# Average Payload Mass for Booster Version F9 v1.1

In [13]:
%sql SELECT AVG("Payload_Mass__kg_") FROM SPACEXTABLE WHERE "Booster_Version" = 'F9 v1.1';

 * sqlite:///my_data1.db
Done.


AVG(Payload_Mass__kg_)
2928.4


In [14]:
# Earliest Success Date on Ground Pad Landing

In [15]:
%sql SELECT MIN(Date) FROM SPACEXTABLE WHERE "Landing_Outcome" = 'Success (ground pad)';

 * sqlite:///my_data1.db
Done.


MIN(Date)
2015-12-22


In [16]:
# Booster versions with successful drone ship landing and payload between 4000 and 6000 kg

In [17]:
%%sql
SELECT "Booster_Version", "Payload_Mass__kg_"
FROM SPACEXTABLE
WHERE "Landing_Outcome" = 'Success (drone ship)'
AND "Payload_Mass__kg_" > 4000
AND "Payload_Mass__kg_" < 6000;

 * sqlite:///my_data1.db
Done.


Booster_Version,PAYLOAD_MASS__KG_
F9 FT B1022,4696
F9 FT B1026,4600
F9 FT B1021.2,5300
F9 FT B1031.2,5200


In [18]:
# Count of each mission outcome

In [19]:

%%sql
SELECT TRIM("Mission_Outcome") AS Normalized_Outcome, COUNT(*) AS Total_Count
FROM SPACEXTABLE
GROUP BY Normalized_Outcome
ORDER BY Total_Count DESC;

 * sqlite:///my_data1.db
Done.


Normalized_Outcome,Total_Count
Success,99
Success (payload status unclear),1
Failure (in flight),1


In [20]:
# Booster versions that carried maximum payload mass

In [21]:
%%sql
SELECT "Booster_Version"
FROM SPACEXTABLE
WHERE "Payload_Mass__kg_" = (
    SELECT MAX("Payload_Mass__kg_")
    FROM SPACEXTABLE
);

 * sqlite:///my_data1.db
Done.


Booster_Version
F9 B5 B1048.4
F9 B5 B1049.4
F9 B5 B1051.3
F9 B5 B1056.4
F9 B5 B1048.5
F9 B5 B1051.4
F9 B5 B1049.5
F9 B5 B1060.2
F9 B5 B1058.3
F9 B5 B1051.6


In [22]:
# List months, failure landing on drone ship, booster version, launch site for year 2015

In [23]:
%%sql
SELECT 
    strftime('%m', "Date") AS Month,
    "Landing_Outcome",
    "Booster_Version",
    "Launch_Site"
FROM SPACEXTABLE
WHERE "Landing_Outcome" = 'Failure (drone ship)' 
AND strftime('%Y', "Date") = '2015';

 * sqlite:///my_data1.db
Done.


Month,Landing_Outcome,Booster_Version,Launch_Site
01,Failure (drone ship),F9 v1.1 B1012,CCAFS LC-40
04,Failure (drone ship),F9 v1.1 B1015,CCAFS LC-40


In [24]:
# Count and rank landing outcomes between two dates

In [25]:
%%sql
SELECT "Landing_Outcome", COUNT(*) AS Total_Count
FROM SPACEXTABLE
WHERE "Date" BETWEEN '2010-06-04' AND '2017-03-20'
GROUP BY "Landing_Outcome"
ORDER BY Total_Count DESC;


 * sqlite:///my_data1.db
Done.


Landing_Outcome,Total_Count
No attempt,10
Success (drone ship),5
Failure (drone ship),5
Success (ground pad),3
Controlled (ocean),3
Uncontrolled (ocean),2
Failure (parachute),2
Precluded (drone ship),1
